## Helper Functions

In [1]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [3]:
import tqdm
import pandas as pd
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Union

In [4]:
%%time

transactions = pd.read_parquet('../input/hm-lgbm-supporting-data/transactions_train.parquet')
customers = pd.read_parquet('../input/hm-lgbm-supporting-data/customers.parquet')
articles = pd.read_parquet('../input/hm-lgbm-supporting-data/articles.parquet')

CPU times: user 2.11 s, sys: 2.67 s, total: 4.78 s
Wall time: 7.95 s


In [5]:
from collections import defaultdict

val_week_purchases_by_cust = defaultdict(list)

val_week_purchases_by_cust.update(
    transactions[transactions.week == transactions.week.max()] \
        .groupby('customer_id')['article_id'] \
        .apply(list) \
        .to_dict()
)

pd.to_pickle(dict(val_week_purchases_by_cust), 'val_week_purchases_by_cust.pkl')

In [6]:
test_week = transactions.week.max()
transactions = transactions[transactions.week != transactions.week.max()]
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [7]:
test_week

104

In [8]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week
28777300,2020-07-08,857913002275398,599580068,0.008458,1,94
28777301,2020-07-08,857913002275398,776237011,0.025407,1,94
28777302,2020-07-08,857913002275398,844294001,0.011847,1,94
28787123,2020-07-08,1658289241058394,877773001,0.007610,1,94
28788562,2020-07-08,3828854365940846,507883009,0.013542,1,94
...,...,...,...,...,...,...
31536744,2020-09-15,18446630855572834764,568601045,0.050831,2,103
31536745,2020-09-15,18446630855572834764,568601045,0.050831,2,103
31536746,2020-09-15,18446630855572834764,898713001,0.067780,2,103
31536747,2020-09-15,18446630855572834764,898713001,0.067780,2,103


## Generating candidates

In [9]:
class GenerateCandidates:
    def __init__(self, transactions, articles, customers):
        self._transactions = transactions
        self._articles = articles
        self._customer = customers
        
    def get_candidates(self):
        return self._last_purchase_candidates, self._bestseller_candidates
    
    def get_bestsellers(self):
        return self._bestseller_previous_week
    
    def generate_candidates(self):
        '''
        1. Last purchase made by each customer
        2. Bestselling articles on weekly basis
        '''
        self._last_purchase_candidates = self.last_purchase_candidates()
        self._bestseller_candidates, self._bestseller_previous_week = self.best_seller_candidates()
        
    def last_purchase_candidates(self):
        '''
        Retrieve Customer Ids with respective weeks of having positive interactions
        '''
        c2weeks = self._transactions.groupby('customer_id')['week'].unique()
        print('#### Customer having positive interactions at week level ####')
        print(c2weeks)
        
        '''
        Generating a dictionary to capture the effect of each purchase made by each customer 
        and then mapping it back to subsequent week of next transaction made
        '''
        c2weeks2shifted_weeks = {}
        for c_id, weeks in c2weeks.items():
            c2weeks2shifted_weeks[c_id] = {}
            for i in range(weeks.shape[0]-1):
                c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
            c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week
        print('\n#### Customer having positive interactions interchanging subsequent purchases ####')
        print(c2weeks2shifted_weeks[272412481300040])
        
        candidates_last_purchase = self._transactions.copy()
        
        '''
        Generate duplicate positive interactions by mapping extracted weeks to subsequent weeks
        of interactions
        '''
        weeks = []
        for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
            weeks.append(c2weeks2shifted_weeks[c_id][week])

        candidates_last_purchase.week=weeks
        print('\n#### Duplicating interactions for each customer to subsequent weeks ####')
        print(candidates_last_purchase[candidates_last_purchase['customer_id']==272412481300040])
        
        return candidates_last_purchase
    
    def best_seller_candidates(self):
        '''
        Retrieve Mean Prices of articles for each week and also rank the highest selling articles
        per week
        '''
        mean_price = self._transactions.groupby(['week', 'article_id'])['price'].mean()
        print('\n#### Mean Price of articles on weekly basis ####')
        print(mean_price)
        
        sales = self._transactions \
                    .groupby('week')['article_id'].value_counts() \
                    .groupby('week').rank(method='dense', ascending=False) \
                    .groupby('week').head(12).rename('bestseller_rank').astype('int8')
        print('\n#### Best seller ranks for articles on weekly basis and based on no of transactions made each week ####')
        print(sales)
        
        bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
        bestsellers_previous_week.week += 1
        print('\n#### Best seller ranks for articles on weekly basis along with mean prices ####')
        print(bestsellers_previous_week)
        
        '''
        Generate best seller candidates for each week and customer combination
        '''
        unique_transactions = self._transactions \
                                .groupby(['week', 'customer_id']) \
                                .head(1) \
                                .drop(columns=['article_id', 'price']) \
                                .copy()
        print('\n#### Unique transactions for each customer on weekly basis ####')
        print(unique_transactions)
        
        candidates_bestsellers = pd.merge(
                                    unique_transactions,
                                    bestsellers_previous_week,
                                    on='week'
                                )
        print('\n#### Best selling Candidates for each week and customer combination ####')
        print(candidates_bestsellers)
        
        '''
        Generate test transactions for each week
        '''
        test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
        test_set_transactions.week = test_week
        
        candidates_bestsellers_test_week = pd.merge(
                                                test_set_transactions,
                                                bestsellers_previous_week,
                                                on='week'
                                            )
        
        '''
        Combining Actuals and test week bestseller candidates
        '''
        candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
        candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)
        print('\n#### Best selling Candidates for customer with ID 272412481300040 ####')
        print(candidates_bestsellers.loc[candidates_bestsellers.customer_id.isin([272412481300040])])
        
        return candidates_bestsellers, bestsellers_previous_week

In [10]:
generate_candidate_instance = GenerateCandidates(transactions, articles, customers)
generate_candidate_instance.generate_candidates()
candidates_last_purchase, candidates_bestsellers = generate_candidate_instance.get_candidates()
bestsellers_previous_week = generate_candidate_instance.get_bestsellers()

#### Customer having positive interactions at week level ####
customer_id
28847241659200          [94, 95, 96, 101, 102]
41318098387474                            [98]
116809474287335                     [101, 103]
200292573348128              [95, 96, 99, 102]
208119717816961                           [94]
                                 ...          
18446590778427270109                 [97, 102]
18446624797007271432                      [95]
18446630855572834764                     [103]
18446662237889060501                     [100]
18446705133201055310                     [102]
Name: week, Length: 439368, dtype: object

#### Customer having positive interactions interchanging subsequent purchases ####
{95: 96, 96: 103, 103: 104}

#### Duplicating interactions for each customer to subsequent weeks ####
              t_dat      customer_id  article_id     price  sales_channel_id  \
29030503 2020-07-15  272412481300040   778064028  0.008458                 1   
29030504 2020-07-15  

In [11]:
candidates_last_purchase

,t_dat,customer_id,article_id,price,sales_channel_id,week
28777300,2020-07-08,857913002275398,599580068,0.008458,1,95
28777301,2020-07-08,857913002275398,776237011,0.025407,1,95
28777302,2020-07-08,857913002275398,844294001,0.011847,1,95
28787123,2020-07-08,1658289241058394,877773001,0.007610,1,104
28788562,2020-07-08,3828854365940846,507883009,0.013542,1,100
...,...,...,...,...,...,...
31536744,2020-09-15,18446630855572834764,568601045,0.050831,2,104
31536745,2020-09-15,18446630855572834764,568601045,0.050831,2,104
31536746,2020-09-15,18446630855572834764,898713001,0.067780,2,104
31536747,2020-09-15,18446630855572834764,898713001,0.067780,2,104


In [12]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-15,272412481300040,1,95,806388001,0.013301
1,2020-07-15,272412481300040,1,95,730683021,0.025643
2,2020-07-15,272412481300040,1,95,610776002,0.008303
3,2020-07-15,272412481300040,1,95,805308002,0.013609
4,2020-07-15,272412481300040,1,95,866383006,0.024971
...,...,...,...,...,...,...
5272411,2020-09-15,18446630855572834764,2,104,918292001,0.041424
5272412,2020-09-15,18446630855572834764,2,104,762846027,0.025104
5272413,2020-09-15,18446630855572834764,2,104,809238005,0.041656
5272414,2020-09-15,18446630855572834764,2,104,673677002,0.024925


In [13]:
bestsellers_previous_week

,week,article_id,bestseller_rank,price
0,95,806388001,1,0.013301
1,95,730683021,2,0.025643
2,95,610776002,3,0.008303
3,95,805308002,4,0.013609
4,95,866383006,5,0.024971
...,...,...,...,...
115,104,918292001,8,0.041424
116,104,762846027,9,0.025104
117,104,809238005,10,0.041656
118,104,673677002,11,0.024925


## Generating Dynamic User Features

In [14]:
class UserFeatures(ABC):
    @abstractmethod
    def get(self) -> pd.DataFrame:
        """
        customer_id -> features
        """
        pass

class AggrFeatures(UserFeatures):
    """
    basic aggregation features(min, max, mean and etc...)
    """
    def __init__(self, transactions_df):
        self.groupby_df = transactions_df.groupby('customer_id', as_index = False)

    def get(self):
        output_df = (
            self.groupby_df['price']
            .agg({
                'mean_transactions': 'mean',
                'max_transactions': 'max',
                'min_transactions': 'min',
                'median_transactions': 'median',
                'sum_transactions': 'sum',
                'max_minus_min_transactions': lambda x: x.max()-x.min()
            })
            .set_index('customer_id')
            .astype('float32')
        )
        return output_df

class CountFeatures(UserFeatures):
    """
    basic features connected with transactions
    """
    def __init__(self, transactions_df, topk = 10):
        self.transactions_df = transactions_df
        self.topk = topk

    def get(self):
        grouped = self.transactions_df.groupby('customer_id', as_index = False)
        #number of transactions, number of online articles,
        #number of transactions bigger than mean price of transactions
        a = (
            grouped
            .agg({
                'article_id': 'count',
                'price': lambda x: sum(np.array(x) > x.mean()),
                'sales_channel_id': lambda x: sum(x == 2),
            })
            .rename(columns = {
                'article_id': 'n_transactions',
                'price': 'n_transactions_bigger_mean',
                'sales_channel_id': 'n_online_articles'
            })
            .set_index('customer_id')
            .astype('int8')
        )
        #number of unique articles, number of store articles
        b = (
            grouped
            .agg({
                'article_id': 'nunique',
                'sales_channel_id': lambda x: sum(x == 1),
            })
            .rename(columns = {
                'article_id': 'n_unique_articles',
                'sales_channel_id': 'n_store_articles',
            })
            .set_index('customer_id')
            .astype('int8')
        )
        #number of transactions that are in top
        topk_articles = self.transactions_df['article_id'].value_counts()[:self.topk].index
        c = (
            grouped['article_id']
            .agg({
               f'top_article_{i}':  lambda x: sum(x == k) for i, k in enumerate(topk_articles)
            }
            )
            .set_index('customer_id')
            .astype('int8')
        )
        
        output_df = a.merge(b, on = ('customer_id')).merge(c, on = ('customer_id'))
        return output_df

# class CustomerFeatures(UserFeatures):
#     """
#     All columns from customers dataframe
#     """
#     def __init__(self, customers_df):
#         self.customers_df = self._prepare_customers(customers_df)
    
#     def _prepare_customers(self, customers_df):
#         customers_df['FN'] = customers_df['FN'].fillna(0).astype('int8')
#         customers_df['Active'] = customers_df['Active'].fillna(0).astype('int8')
#         customers_df['club_member_status'] = customers_df['club_member_status'].fillna('UNKNOWN')
#         customers_df['age'] = customers_df['age'].fillna(customers_df['age'].mean()).astype('int8')
#         customers_df['fashion_news_frequency'] = (
#             customers_df['fashion_news_frequency']
#             .replace('None', 'NONE')
#             .replace(np.nan, 'NONE')
#         )
#         return customers_df

#     def get(self):
#         output = (
#             self.customers_df[filter(lambda x: x != 'postal_code', customers_df.columns)]
#             .set_index('customer_id')
#         )
#         return output

class ArticlesFeatures(UserFeatures):
    """
    returns article features: whether category appears in top categories
    """
    def __init__(self, transactions_df, articles, topk = 10):
        self.merged_df = transactions_df.merge(articles, on = ('article_id'))
        self.articles = articles
        self.topk = topk
    
    def get(self):
        output_df = None

        for col in tqdm.tqdm(self.articles.columns, desc = 'extracting features'):
            if 'name' in col:
                if output_df is None:
                    output_df = self.aggregate_topk(self.merged_df, col, self.topk)
                else:
                    intermediate_out = self.aggregate_topk(self.merged_df, col, self.topk)
                    output_df = output_df.merge(intermediate_out, on = ('customer_id'))
        return output_df

    def return_value_counts(self, df, column_name, k):
        value_counts = df[column_name].value_counts()[:k].index
        value_counts = list(map(lambda x: x[1], value_counts))
        return value_counts

    def aggregate_topk(self, merged_df, column_name, k):
        grouped_df_indx = merged_df.groupby('customer_id')
        grouped_df = merged_df.groupby('customer_id', as_index = False)
        
        topk_values = self.return_value_counts(grouped_df_indx, column_name, k)
        #how many transactions appears in top category(column)
        n_top_k = (
            grouped_df[column_name]
            .agg({
                f'top_{column_name}_{i}': lambda x: sum(x == k) for i, k in enumerate(topk_values)
            })
            .set_index('customer_id')
            #.astype('int16')
        )
        return n_top_k

class UserFeaturesCollector:
    """
    collect all features and aggregate them
    """
    @staticmethod
    def collect(features: Union[List[UserFeatures], List[str]], **kwargs) -> pd.DataFrame:
        output_df = None

        for feature in tqdm.tqdm(features):
            if isinstance(feature, UserFeatures):
                feature_out = feature.get(**kwargs)
            if isinstance(feature, str):
                try:
                    feature_out = pd.read_csv(feature)
                except:
                    feature_out = pd.read_parquet(feature)

            if output_df is None:
                output_df = feature_out
            else:
                output_df = output_df.merge(feature_out, on = ('customer_id'))
        return output_df

In [15]:
customer_features = UserFeaturesCollector.collect([
    AggrFeatures(transactions),
    CountFeatures(transactions, 3),
    # CustomerFeatures(customers),
    ArticlesFeatures(transactions, articles, 3),
])

100%|██████████| 3/3 [31:02<00:00, 620.69s/it]


## Combining transactions and candidates / negative examples

In [16]:
transactions['purchased'] = 1

In [17]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [18]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
28777300,2020-07-08,857913002275398,599580068,0.008458,1,94,1.0
28777301,2020-07-08,857913002275398,776237011,0.025407,1,94,1.0
28777302,2020-07-08,857913002275398,844294001,0.011847,1,94,1.0
28787123,2020-07-08,1658289241058394,877773001,0.007610,1,94,1.0
28788562,2020-07-08,3828854365940846,507883009,0.013542,1,94,1.0
...,...,...,...,...,...,...,...
5272411,2020-09-15,18446630855572834764,918292001,0.041424,2,104,0.0
5272412,2020-09-15,18446630855572834764,762846027,0.025104,2,104,0.0
5272413,2020-09-15,18446630855572834764,809238005,0.041656,2,104,0.0
5272414,2020-09-15,18446630855572834764,673677002,0.024925,2,104,0.0


In [19]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [20]:
data.shape

(18475436, 7)

In [21]:
data.purchased.mean()

0.1369278646522875

### Add bestseller information

In [22]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [23]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [24]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [25]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [26]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [27]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [28]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [29]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 328 ms, sys: 239 ms, total: 567 ms
Wall time: 561 ms


# Model training

In [30]:
from lightgbm.sklearn import LGBMRanker

In [31]:
optimized_params = {'objective': "lambdarank",
                    'metric': "ndcg",
                    'num_threads': 4,
                    'boosting_type': "dart",
                    'importance_type': "gain",
                    'verbose': -1,
                    'n_estimators': 100, 
                    'learning_rate': 0.26762621693602034, 
                    'num_leaves': 90, 
                    'max_depth': 9, 
                    'min_data_in_leaf': 10, 
                    'lambda_l1': 35, 
                    'lambda_l2': 10, 
                    'min_gain_to_split': 7.702426339836351, 
                    'bagging_fraction': 0.30000000000000004, 
                    'bagging_freq': 1, 
                    'feature_fraction': 0.7,
                    'seed': 42}

In [32]:
# ranker = LGBMRanker(
#     objective="lambdarank",
#     metric="ndcg",
#     boosting_type="dart",
#     n_estimators=1,
#     importance_type='gain',
#     verbose=10
# )

ranker = LGBMRanker(
            random_state=42, 
            **optimized_params
        )

In [33]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=35, reg_alpha=0.0 will be ignored. Current value: lambda_l1=35
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=7.702426339836351, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.702426339836351
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda

In [34]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.907099131677047
article_id 0.036272789899231606
product_type_no 0.01723738924354976
department_no 0.011527003617157696
garment_group_no 0.008661112879279242
colour_group_code 0.0055453306130126894
section_no 0.004047105246273704
graphical_appearance_no 0.0034418029810460195
perceived_colour_value_id 0.0031542629063174067
perceived_colour_master_id 0.0015964930383221025
index_code 0.000971793350211611
index_group_no 0.0004301697593233198
age 1.5614789227798833e-05
fashion_news_frequency 0.0
club_member_status 0.0
Active 0.0
FN 0.0
postal_code 0.0


# Calculate predictions

In [35]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.34 µs


# Evaluate predictions

In [36]:
apks = []

for c_id, gt in val_week_purchases_by_cust.items():
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    apks.append(apk(gt, pred[:12]))

np.mean(apks)

0.0237936920883126